In [41]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity


import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

In [7]:
df_optimizado = pd.read_parquet('..//Datasets//df_tratamientoML.parquet')

In [8]:
# Obtener los valores únicos en la columna 'user_id'
unique_users = df_optimizado['user_id'].unique()

# Obtener los valores únicos en la columna 'item_name'
unique_items = df_optimizado['item_name'].unique()

print("Valores únicos en 'user_id':")
print(unique_users)

print("\nValores únicos en 'item_name':")
print(unique_items)

Valores únicos en 'user_id':
['76561197970982479' 'js41637' 'evcentric' ... '76561198323066619'
 '76561198326700687' '76561198329548331']

Valores únicos en 'item_name':
['Counter-Strike' 'Team Fortress Classic' 'Day of Defeat' ... 'ChaosTower'
 'Aveyond 4: Shadow Of The Mist' 'Arachnophobia']


In [11]:
mapeo_user_id = {}
contador = 1

for user_id in unique_users:
    mapeo_user_id[user_id] = f'user{contador}'
    contador += 1

# Reemplazar los valores numéricos en 'user_id' con los identificadores de usuario
df_optimizado['user_id'] = df_optimizado['user_id'].map(mapeo_user_id)

# Verificar el DataFrame actualizado
print(df_optimizado.head())

  user_id                  item_name    rating
0   user1             Counter-Strike  0.100000
1   user1      Team Fortress Classic  0.000000
2   user1              Day of Defeat  0.116667
3   user1         Deathmatch Classic  0.000000
4   user1  Half-Life: Opposing Force  0.000000


In [13]:
# Agrupar por 'user_id' y calcular el promedio del rating
df_usuario_promedio = df_optimizado.groupby('user_id')['rating'].mean().reset_index()

# Verificar el nuevo DataFrame con los promedios de rating por usuario
print(df_usuario_promedio.head())

     user_id     rating
0      user1  12.037004
1     user10  43.925797
2    user100  52.277632
3   user1000  46.967279
4  user10000   4.915116


In [15]:
# Obtener los valores únicos en la columna 'user_id'
unique_users = df_optimizado['user_id'].unique()

# Obtener los valores únicos en la columna 'item_name'
unique_items = df_optimizado['item_name'].unique()

print("Valores únicos en 'user_id':")
print(unique_users)

print("\nValores únicos en 'item_name':")
print(unique_items)

Valores únicos en 'user_id':
['user1' 'user2' 'user3' ... 'user70910' 'user70911' 'user70912']

Valores únicos en 'item_name':
['Counter-Strike' 'Team Fortress Classic' 'Day of Defeat' ... 'ChaosTower'
 'Aveyond 4: Shadow Of The Mist' 'Arachnophobia']


In [16]:
# Crear un diccionario de mapeo para asignar números a los valores únicos en 'user_id'
mapeo_user_id = {}
contador = 1

for user_id in unique_users:
    mapeo_user_id[user_id] = contador
    contador += 1

# Reemplazar los valores en 'user_id' con los números correspondientes
df_optimizado['user_id'] = df_optimizado['user_id'].map(mapeo_user_id)

# Verificar el DataFrame actualizado
print(df_optimizado.head())

   user_id                  item_name    rating
0        1             Counter-Strike  0.100000
1        1      Team Fortress Classic  0.000000
2        1              Day of Defeat  0.116667
3        1         Deathmatch Classic  0.000000
4        1  Half-Life: Opposing Force  0.000000


In [38]:
df_optimizado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10947 entries, 0 to 5086621
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    10947 non-null  int64  
 1   item_name  10947 non-null  object 
 2   rating     10947 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 600.1+ KB


In [39]:
# Pivotar el DataFrame
user_item_matrix = df_optimizado.pivot(index='user_id', columns='item_name', values='rating')

# Mostrar el nuevo DataFrame
print(user_item_matrix)


item_name   神明的一天世界(god's one day world)  \
user_id                                    
1                                    NaN   
2                                    NaN   
3                                    NaN   
4                                    NaN   
5                                    NaN   
...                                  ...   
60636                                NaN   
68529                                NaN   
69212                                NaN   
69595                                NaN   
70522                                NaN   

item_name  ! that bastard is trying to steal our gold !  \
user_id                                                   
1                                                   NaN   
2                                                   NaN   
3                                                   NaN   
4                                                   NaN   
5                                                   NaN   
...           

In [40]:

# Calcular la media de las calificaciones por usuario
user_means = user_item_matrix.mean(axis=1)

# Calcular la diferencia entre el valor máximo y mínimo de las calificaciones por usuario
user_max = user_item_matrix.max(axis=1)
user_min = user_item_matrix.min(axis=1)
user_diff = user_max - user_min

# Restar la media y dividir por la diferencia
normalized_user_item_matrix = (user_item_matrix.sub(user_means, axis=0)).div(user_diff, axis=0)

# Eliminar usuarios con una sola calificación o calificaciones idénticas
normalized_user_item_matrix.dropna(axis=0, thresh=2, inplace=True)

# Mostrar el nuevo DataFrame normalizado
print(normalized_user_item_matrix)

item_name   神明的一天世界(god's one day world)  \
user_id                                    
1                                    NaN   
2                                    NaN   
3                                    NaN   
4                                    NaN   
5                                    NaN   
...                                  ...   
34650                                NaN   
37231                                NaN   
38490                                NaN   
41493                                NaN   
42956                                NaN   

item_name  ! that bastard is trying to steal our gold !  \
user_id                                                   
1                                                   NaN   
2                                                   NaN   
3                                                   NaN   
4                                                   NaN   
5                                                   NaN   
...           

In [42]:
# Supongamos que tienes la matriz normalizada 'normalized_user_item_matrix'

# Convertir a matriz dispersa (sparse matrix)
sparse_matrix = csr_matrix(normalized_user_item_matrix.values)

# Ahora 'sparse_matrix' contiene la matriz dispersa



In [45]:
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

NameError: name 'piv' is not defined

In [44]:

# Convertir a matriz dispersa (sparse matrix)
piv_sparse = csr_matrix(user_item_matrix.values)

piv_norm.fillna(0, inplace=True)
# Calcular la similitud entre juegos (item_similarity)
item_similarity = cosine_similarity(piv_sparse)

# Calcular la similitud entre usuarios (user_similarity)
user_similarity = cosine_similarity(piv_sparse.T)

# Ahora tienes las matrices de similitud listas para su uso en tu modelo de recomendación

ValueError: Input contains NaN.